In [1]:
import json
import numpy as np
from sort import *
import os
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import shutil

In [2]:
PREDICTIONS_FILENAME = '/Users/darylwilding-mcbride/Downloads/result.json'
TEST_DIR = '/Users/darylwilding-mcbride/Downloads/yolo-train-rt-1000-4200-23-may/test/'
FEATURE_DIR = '/Users/darylwilding-mcbride/Downloads/tracked-features'
PIXELS_HEIGHT = PIXELS_WIDTH = 910

In [3]:
with open(PREDICTIONS_FILENAME, 'r') as f:
    predictions = json.load(f)

In [4]:
predictions[0]['objects'][0]

{'class_id': 1,
 'name': 'charge-2',
 'relative_coordinates': {'center_x': 0.165364,
  'center_y': 0.619003,
  'width': 0.152393,
  'height': 0.131943},
 'confidence': 0.852287}

In [5]:
def convert_rel_coords(rel_cx,rel_cy,rel_width,rel_height):
    width = rel_width * PIXELS_WIDTH
    height = rel_height * PIXELS_HEIGHT
    cx = rel_cx * PIXELS_WIDTH
    cy = rel_cy * PIXELS_HEIGHT
    x1 = int(cx - (width / 2))
    y1 = int(cy - (height / 2))
    x2 = int(cx + (width / 2))
    y2 = int(cy + (height / 2))
    return (x1,y1,x2,y2)

In [6]:
mot_tracker = Sort()

In [7]:
# initialise the directories required for the data set creation
if os.path.exists(FEATURE_DIR):
    shutil.rmtree(FEATURE_DIR)
os.makedirs(FEATURE_DIR)


In [8]:
# load the font to use for labelling the overlays
feature_label_font = ImageFont.truetype('/Library/Fonts/Arial.ttf', 10)

In [9]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]


In [10]:
for frame_idx in range(len(predictions)):
# for frame_idx in range(1):
    filename = os.path.basename(predictions[frame_idx]['filename'])
    detections = predictions[frame_idx]['objects']
    objs = []
    for detection_idx in range(len(detections)):
        rel_coords = detections[detection_idx]['relative_coordinates']
        abs_coords = convert_rel_coords(rel_coords['center_x'],rel_coords['center_y'],rel_coords['width'],rel_coords['height'])
        score = detections[detection_idx]['confidence']
        objs.append((abs_coords[0], abs_coords[1], abs_coords[2], abs_coords[3], score))
    objs_array = np.array(objs)  # convert to a numpy array for the SORT call
    track_ids = mot_tracker.update(objs_array)
    track_ids_l = list(map(tuple, track_ids))
    image = Image.open('{}/{}'.format(TEST_DIR, filename))
    draw = ImageDraw.Draw(image)
    for track_id in track_ids_l:
        x0 = int(track_id[0])
        y0 = int(track_id[1])
        x1 = int(track_id[2])
        y1 = int(track_id[3])
        feature_id = int(track_id[4])
        feature_colour = tableau20[feature_id % len(tableau20)]
        draw.rectangle(xy=[(x0, y0), (x1, y1)], fill=None, outline=feature_colour)
        draw.text((x0, y0-12), "feature {}".format(feature_id), font=feature_label_font, fill=feature_colour)
    image.save('{}/{}'.format(FEATURE_DIR, filename))